In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [30]:
df = pd.read_csv('Animation.csv')

In [31]:
df.head()

,animeID,name,source,producers,genre,studio,episodes,airing,rank,popularity,members,favorites,reviewers,Animation_score
0,1,Cowboy Bebop,Original,Bandai Visual,Action,Sunrise,26.0,False,26,39,795733,43460,405664,8.81
1,5,Cowboy Bebop: Tengoku no Tobira,Original,Sunrise,Action,Bones,1.0,False,164,449,197791,776,120243,8.41
2,6,Trigun,Manga,Victor Entertainment,Action,Madhouse,26.0,False,255,146,408548,10432,212537,8.30
3,7,Witch Hunter Robin,Original,Bandai Visual,Action,Sunrise,26.0,False,2371,1171,79397,537,32837,7.33
4,8,Bouken Ou Beet,Manga,TV Tokyo,Adventure,Toei Animation,52.0,False,3544,3704,11708,14,4894,7.03


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13631 entries, 0 to 13630
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   animeID          13631 non-null  int64  
 1   name             13631 non-null  object 
 2   source           13631 non-null  object 
 3   producers        7414 non-null   object 
 4   genre            13569 non-null  object 
 5   studio           8369 non-null   object 
 6   episodes         13349 non-null  float64
 7   airing           13631 non-null  bool   
 8   rank             13631 non-null  int64  
 9   popularity       13631 non-null  int64  
 10  members          13631 non-null  int64  
 11  favorites        13631 non-null  int64  
 12  reviewers        13631 non-null  int64  
 13  Animation_score  13518 non-null  float64
dtypes: bool(1), float64(2), int64(6), object(5)
memory usage: 1.4+ MB


In [33]:
# Using classification to determain genre for null values

        # PREPROCCESING
columsPredictGenre = ['source','producers','studio','episodes','rank','popularity','members','airing','favorites','reviewers','Animation_score']

# Since Score has a little outlier and those who don'nt have score also don't have genre,
# This won't effect to predict genre
dfG = df[df.Animation_score.notna() == True]
yG = dfG['genre'].fillna('other genre')
dfG = dfG[columsPredictGenre]
categorical_columns = dfG.select_dtypes(include=['object']).columns

numerical_columns = dfG.select_dtypes(include=['number']).columns

dfG[categorical_columns] = dfG[categorical_columns].fillna('Other')

# Fill null values in numerical columns with 0
dfG[numerical_columns] = dfG[numerical_columns].fillna(0)

# Check for null values
yG.isnull().any().any()

False

In [34]:
#dfg_encoded = pd.get_dummies(dfG, columns=['source','producers','studio'])

    # Select top categories to reduce the hot code encoding

def one_hot_encode_top(dfG, column_name,numb):
    top_categories = dfG[column_name].value_counts().index[:numb]
    dfG[column_name] = dfG[column_name].apply(lambda x: x if x in top_categories else 'Other')
    df_encoded = pd.get_dummies(dfG, columns=[column_name])
    return df_encoded

dfG = one_hot_encode_top(dfG, 'source',4)
dfG = one_hot_encode_top(dfG, 'producers',12)
dfG = one_hot_encode_top(dfG, 'studio',12)
dfG = pd.get_dummies(dfG, columns=['airing'])


dfG


,episodes,rank,popularity,members,favorites,reviewers,Animation_score,source_Game,source_Manga,source_Original,...,studio_OLM,studio_Other,studio_Production I.G,studio_Studio Deen,studio_Studio Pierrot,studio_Sunrise,studio_TMS Entertainment,studio_Toei Animation,airing_False,airing_True
0,26.0,26,39,795733,43460,405664,8.81,0,0,1,...,0,0,0,0,0,1,0,0,1,0
1,1.0,164,449,197791,776,120243,8.41,0,0,1,...,0,1,0,0,0,0,0,0,1,0
2,26.0,255,146,408548,10432,212537,8.30,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,26.0,2371,1171,79397,537,32837,7.33,0,0,1,...,0,0,0,0,0,1,0,0,1,0
4,52.0,3544,3704,11708,14,4894,7.03,0,1,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13553,1.0,10776,15213,38,0,21,2.57,0,0,1,...,0,1,0,0,0,0,0,0,1,0
13554,1.0,11138,15293,29,0,17,4.35,0,0,1,...,0,1,0,0,0,0,0,0,1,0
13556,1.0,13398,15346,19,0,7,4.57,0,0,1,...,0,1,0,0,0,0,0,0,1,0
13590,40.0,11345,15227,36,0,2,5.50,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [35]:
# Modling for predicting genre using random forest
X_train, X_test, y_train, y_test = train_test_split(dfG, yG, test_size=0.2, random_state=42)

# Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=400, random_state=42)

# Train
rf_classifier.fit(X_train, y_train)

# Predict
predictions = rf_classifier.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f'Accuracy: {accuracy}')
print(report)

Accuracy: 0.4282544378698225
               precision    recall  f1-score   support

       Action       0.44      0.74      0.55       631
    Adventure       0.40      0.28      0.33       259
         Cars       0.33      0.11      0.17         9
       Comedy       0.42      0.60      0.49       532
     Dementia       0.39      0.45      0.42        51
       Demons       0.00      0.00      0.00         7
        Drama       0.37      0.19      0.25       154
        Ecchi       0.50      0.12      0.20        16
      Fantasy       0.29      0.10      0.15        99
         Game       0.45      0.19      0.27        26
        Harem       0.00      0.00      0.00        14
   Historical       0.04      0.02      0.03        53
       Horror       0.00      0.00      0.00        10
         Kids       0.46      0.43      0.45       166
        Magic       0.00      0.00      0.00        15
 Martial Arts       0.00      0.00      0.00         1
        Mecha       0.75      0.14 

C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NameError: name 'null' is not defined